<a href="https://colab.research.google.com/github/rising1/BingImageDownloader/blob/master/Heartbeat2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#a = []
#while(1):
#   a.append(1)

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!pip install google_images_download

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
     |████████████████████████████████| 911kB 4.7MB/s 
  Created wheel for google-images-download: filename=google_images_download-2.8.0-py2.py3-none-any.whl size=14550 sha256=481564dbd02c166217d1736f85173b67c6eec9e99a3199c6998387a0428ef51c
  Stored in directory: /root/.cache/pip/wheels/1f/28/ad/f56e7061e1d2a9a1affe2f9c649c2570cb9198dd24ede0bbab
Successfully built google-images-download


In [0]:
#!git clone https://github.com/rising1/Heartbeat.git
#%cd Heartbeat
#%ls -l

In [0]:
# memory footprint support libraries/code
#!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
#!pip install gputil
#!pip install psutil
#!pip install humanize
#import psutil
#import humanize
#import os
#import GPUtil as GPU
#GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
#gpu = GPUs[0]
#def printm():
# process = psutil.Process(os.getpid())
# print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
# print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))


In [0]:
# Import needed packages
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import time
import os, csv
import glob
from torchvision import transforms, datasets
from PIL import Image
from torch.utils.data import DataLoader
import io
from matplotlib import pyplot as plt
import numpy as np


class test_images():

    def __init__(self,batch_size,shuffle_data):
        global data_transform
        self.batch_sizes = batch_size
        self.shuffle_data = shuffle_data
        data_transform = transforms.Compose([
                    transforms.Resize(128),
                    # transforms.CenterCrop(72),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])


    def get_tensor(self,image_bytes):
        image = Image.open(io.BytesIO(image_bytes))
        return data_transform(image).unsqueeze(0)


    def data_transformation(self,image):
        global data_transform
        print("type(image)=",type(image))
        image_dataset = datasets.ImageFolder(image, data_transform)
        return image_dataset

    def eval_test(self,path_to_images):
        self.dir_path = path_to_images
        self.image_dataset = datasets.ImageFolder(self.dir_path+'/eval',data_transform)
        self.dataloaders = torch.utils.data.DataLoader(self.image_dataset,
                            batch_size=self.batch_sizes,
                            shuffle=self.shuffle_data,
                            num_workers=0)

        #print(type(self.dataloaders["train"][0]))
        self.dataset_sizes = len(self.image_dataset)
        return self.dataloaders

    def getitem(self, index):
        return self.image_dataset.__getitem__(index)   # return image path

 

In [0]:
def test_model(model,my_test_loader,validate_path):
    bird_list = ['Owl', 'Bittern', 'Blackbird', 'Tit', 'Chicken', 'Parakeet', 'Peregrine', 'Dove', 'Plover',
                 'Puffin', 'Robin', 'sparrowhawk']
    my_test_loader_eval = my_test_loader.eval_test(dataPathRoot)
    model.eval()
    test_acct = 0.0
    test_history = []
    image_list = []
    label_list = []
    predictions_list = []
    images, labels = next(iter(my_test_loader_eval))
    if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
    #  Predict classes using images from the test set
    outputs = model(images)
    _, prediction = torch.max(outputs.data, 1)
    for image in images:
        image = image.cpu()
        image_list.append(imshow(image))
    for i in range(len(prediction)):
        if (birds_listing(validate_path)[int(prediction[i].cpu().numpy())]) == (
                bird_list[labels.data[i].cpu().numpy()]):
                tick = "Yes"    # str(u'\2714'.encode('utf-8')) # approval tick mark
        else:
                tick = "No"     # str(u'\2717'.encode('utf-8')) # cross mark
        predictions_list.append(birds_listing(validate_path)[int(prediction[i].cpu().numpy())]  +
                                "\n" + "\n" + "       " + tick)
                                # "\n" + bird_list[labels.data[i].cpu().numpy()] + " " + tick)
    show_images(image_list,2,predictions_list)

def show_images(images, cols=1, titles=None):
    """Display a list of images in a single figure with matplotlib.

    Parameters
    ---------
    images: List of np.arrays compatible with plt.imshow.

    cols (Default = 1): Number of columns in figure (number of rows is
                        set to np.ceil(n_images/float(cols))).

    titles: List of titles corresponding to each image. Must have
            the same length as titles.
    """
    assert ((titles is None) or (len(images) == len(titles)))
    n_images = len(images)
    if titles is None: titles = ['Image (%d)' % i for i in range(1, n_images + 1)]
    fig = plt.figure()
    for n, (image, title) in enumerate(zip(images, titles)):
        a = fig.add_subplot(cols, np.ceil(n_images / float(cols)), n + 1)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(image)
        a.set_title(title,fontsize=8)
    #  fig.set_size_inches(np.array(fig.get_size_inches()) * n_images)
    #  fig = plt.figure(figsize=(6, 3))
    plt.show(block=False)
    plt.pause(8)
    plt.close()

def imshow(inp, title=None):
    """Imshow for Tensor."""
    image_list = []
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    return inp

def birds_listing(validate_path):
    with open(validate_path,'r') as f:
       #  with open('C:/Users/phfro/Documents/python/data/Class_validate.txt', 'r') as f:
       #  with open('/content/drive/My Drive/Colab Notebooks/Class_validate.txt', 'r') as f:
       reader = csv.reader(f)
       classes = list(reader)[0]
       classes.sort()
       #  self.classes = open('/content/drive/My Drive/Colab Notebooks/Class_validate.txt').read()
       #  print("self.classes=",classes)
       #  print("len self.classes=",len(classes))
    return classes

In [0]:
# transforms to apply to the datan
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
import torch
import csv
import os


class HawkLoader:
    def __init__(self, dir_path, batch_sizes, pic_size, computer):
        self.batch_sizes = batch_sizes
        self.dir_path = dir_path
        self.pic_size = pic_size
        data_transforms = {
            'train': transforms.Compose([
                transforms.Resize(224),
                transforms.RandomResizedCrop(self.pic_size),
                # transforms.CenterCrop(self.pic_size),
                # transforms.RandomResizedCrop(self.pic_size),
                # transforms.CenterCrop(self.pic_size),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
            'val': transforms.Compose([
                transforms.Resize(224),
                transforms.CenterCrop(self.pic_size),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
            'test': transforms.Compose([
                transforms.Resize(224),
                transforms.CenterCrop(self.pic_size),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
        }

        # transforms.RandomResizedCrop(120,(1,1),(1,1),2),
        # print(os.path.join(self.dir_path, 'train'))
        if (computer == "home_laptop" or computer == "home_red_room" ):
            image_datasets = {x: datasets.ImageFolder(os.path.join(self.dir_path, x),
                              data_transforms[x]) for x in ['train', 'val', 'test']}
        elif computer == "work":
            image_datasets = {x: datasets.ImageFolder(os.path.join('D:/', x),
                              data_transforms[x]) for x in ['train', 'val', 'test']}
        self.dataloaderTrain = {x: torch.utils.data.DataLoader(
                            image_datasets[x],
                            batch_size=self.batch_sizes,
                            shuffle=True, num_workers=4)
                       for x in ['train']}
        self.dataloaderTest = {x: torch.utils.data.DataLoader(
                            image_datasets[x],
                            batch_size=self.batch_sizes,
                            shuffle=False, num_workers=4)
                       for x in [ 'val', 'test']}
        #print(type(self.dataloaders["train"][0]))
        self.dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}

        #  self.classes = open('BirdList.txt').read().splitlines()
        #  self.classesTest = ('buzzard', 'golden eagle','kestrel', 'peregrine falcon',
        #                    'red kite', 'sparrow hawk')

    def birds_listing(self):
        with open('/content/drive/My Drive/Colab Notebooks/Class_validate.txt', 'r') as f:
           reader = csv.reader(f)
           self.classes = list(reader)[0]
           self.classes.sort()
           #  self.classes = open('/content/drive/My Drive/Colab Notebooks/Class_validate.txt').read()
           #  print("self.classes=",self.classes)
           #  print("len self.classes=",len(self.classes))
        return self.classes

    def cifar10_train_loader(self):
        # Define transformations for the training set, flip the images randomly, crop out and apply mean and std normalization
        train_transformations = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(32, padding=4),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

        # Load the training set
        train_set = CIFAR10(root="./data", train=True, transform=train_transformations, download=True)

        # Create a loder for the training set
        train_loader = DataLoader(train_set, batch_size=32, shuffle=True, num_workers=0)
        print ("returning train loader object")
        return train_loader

    def cifar10_test_loader(self):
        # Define transformations for the test set
        test_transformations = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

        ])

        # Load the test set, note that train is set to False
        test_set = CIFAR10(root="./data", train=False, transform=test_transformations, download=True)

        # Create a loder for the test set, note that both shuffle is set to false for the test loader
        test_loader = DataLoader(test_set, batch_size=32, shuffle=False, num_workers=0)
        return test_loader

In [0]:
# Import needed packages
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import time
import os, csv
import glob

# Hyper-parameters
colour_channels = 3  # used in SimpleNet
no_feature_detectors = 128 # used in Unit
kernel_sizes = 3  # used in Unit
stride_pixels = 1  # used in Unit
padding_pixels = 1  # used in Unit
pooling_factor = 2  # used in SimpleNet
pic_size = 128 # used in SimpleNet
output_classes = 220  # used in SimpleNet
learning_rate = 0.0001  # used in HeartbeatClean
decay_cycles = 1  # default to start
weight_decay = 0.0001  # used in HeartbeatClean
dropout_factor = 0.0  # used in Unit
flattener = 64
faff = 'false'
linear_mid_layer = 1024
num_epochs = 200  # used in HeartbeatClean
snapshot_points = num_epochs / 1
batch_sizes = 48 # used in HeartbeatClean
#  batch_sizes = 6 # used in HeartbeatClean
loadfile = True

validate_path = '/content/drive/My Drive/Colab Notebooks/Class_validate.txt'
dataPathRoot = '/content/drive/My Drive/Colab Notebooks'
# dataPathRoot = 'C:/Users/phfro/PycharmProjects/Heartbeat'
#dataPathRoot = 'E:/'
# validate_path = 'C:/Users/phfro/PycharmProjects/Heartbeat/Class_validate.txt'
#alidate_path = 'E:/Class_validate.txt'
computer = "home_laptop"
# deploy_test = Hawknet_Depld.test_images(12, False)
# Check if gpu support is available
cuda_avail = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Args lists to pass through to models
UnitArgs = [kernel_sizes, stride_pixels, padding_pixels]
SimpleNetArgs = [UnitArgs, dropout_factor,output_classes, 
                 colour_channels, no_feature_detectors, 
                 pooling_factor]


class Unit(nn.Module):
    def __init__(self, UnitArgs, in_channel, out_channel):
        
        super(Unit, self).__init__()
        self.conv = nn.Conv2d( kernel_size = UnitArgs[0], stride = UnitArgs[1],
                               padding = UnitArgs[2],
                               in_channels = in_channel, out_channels = out_channel)
        self.bn = nn.BatchNorm2d(num_features=out_channel)
        # self.do = nn.Dropout(dropout_factor)
        self.relu = nn.ReLU()


    def forward(self, input):
        output = self.conv(input)
        output = self.bn(output)
        # output = self.do(output)
        output = self.relu(output)
        return output


class SimpleNet(nn.Module):
    def __init__(self, SimpleNetArgs):
        super(SimpleNet, self).__init__()

        # Break out the parameters for the model
        UnitArgs = SimpleNetArgs[0]
        dropout_factor = SimpleNetArgs[1]
        output_classes = SimpleNetArgs[2]
        colour_channels = SimpleNetArgs[3]
        no_feature_detectors = SimpleNetArgs[4]
        pooling_factor = SimpleNetArgs[5]


        # Create 14 layers of the unit with max pooling in between
        self.unit1 = Unit(UnitArgs,colour_channels, no_feature_detectors)
        self.unit2 = Unit(UnitArgs,no_feature_detectors, no_feature_detectors)
        self.unit3 = Unit(UnitArgs,no_feature_detectors, no_feature_detectors)

        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.unit4 = Unit(UnitArgs,no_feature_detectors, no_feature_detectors * 2)
        self.unit5 = Unit(UnitArgs,no_feature_detectors * 2, no_feature_detectors * 2)
        self.unit6 = Unit(UnitArgs,no_feature_detectors * 2, no_feature_detectors * 2)
        self.unit7 = Unit(UnitArgs,no_feature_detectors * 2, no_feature_detectors * 2)

        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.unit8 = Unit(UnitArgs,no_feature_detectors * 2, no_feature_detectors * 4)
        self.unit9 = Unit(UnitArgs,no_feature_detectors * 4, no_feature_detectors * 4)
        self.unit10 = Unit(UnitArgs,no_feature_detectors * 4, no_feature_detectors * 4)
        self.unit11 = Unit(UnitArgs,no_feature_detectors * 4, no_feature_detectors * 4)

        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.unit12 = Unit(UnitArgs,no_feature_detectors * 4, no_feature_detectors * 4)
        self.unit13 = Unit(UnitArgs,no_feature_detectors * 4, no_feature_detectors * 4)
        self.unit14 = Unit(UnitArgs,no_feature_detectors * 4, no_feature_detectors * 4)

        self.avgpool = nn.AvgPool2d(kernel_size=4)

        # Add all the units into the Sequential layer in exact order
        self.net = nn.Sequential(self.unit1, self.unit2, self.unit3, self.pool1, self.unit4, self.unit5, self.unit6
                                 , self.unit7, self.pool2, self.unit8, self.unit9, self.unit10, self.unit11, self.pool3,
                                 self.unit12, self.unit13, self.unit14, self.avgpool)
        self.fc = nn.Linear(no_feature_detectors * flattener, output_classes)
        #self.fc = nn.Linear(no_feature_detectors * 4 * 4, output_classes)
        # self.fc = nn.Linear(no_feature_detectors * 4 , output_classes)
        # self.fc = nn.Linear(int (no_feature_detectors / 4), output_classes)
        # self.fc_final = nn.Linear(linear_mid_layer, output_classes)

    def forward(self, input):
        output = self.net(input)
        # print("net(input) ",output.shape)
        output = output.view(-1, no_feature_detectors * flattener )
        #output = output.view(-1, no_feature_detectors * 4 * 4)
        # output = output.view(-1, no_feature_detectors * 4 )
        # output = output.view(-1, int(no_feature_detectors / 4))
        #print("output.view ",output.shape)
        output = self.fc(output)
        #output = self.fc_final(output)
        #print("fc(output) ",output.shape)
        return output


def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']


def first_learning_rate(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        print("learning rate adjusted to ", lr)


def lr_decay_cycles(cycles):
    global decay_cycles
    decay_cycles = cycles
    return decay_cycles


def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every decay epochs"""
    global decay_cycles
    learning_rate = get_lr(optimizer) * (0.1 ** (epoch // decay_cycles))
    for param_group in optimizer.param_groups:
        param_group['lr'] = learning_rate


def get_latest_file(path, *paths):
    """Returns the name of the latest (most recent) file
    of the joined path(s)"""
    fullpath = os.path.join(path, *paths)
    list_of_files = glob.glob(fullpath)  # You may use iglob in Python3
    if not list_of_files:                # I prefer using the negation
        return None                      # because it behaves like a shortcut
    latest_file = max(list_of_files, key=os.path.getctime)
    _, filename = os.path.split(latest_file)
    return filename

def load_latest_saved_model(chosen_model = None,is_eval = False):
    global dataPathRoot, loadfile, model, optimizer, \
            epoch, loss, device
    # load a saved model if one exists
    comp_root = dataPathRoot + "/saved_models/"

    if chosen_model is not None:
        selected_model = chosen_model
        print("looking for ",comp_root + selected_model)
        print("exists = ",os.path.isfile(comp_root + selected_model))
    else:
        stub_name = "Birdies_model_*"
        selected_model = get_latest_file(comp_root, stub_name)
        print("latest filename=", selected_model)

    if os.path.isfile(comp_root + selected_model) and loadfile == True:
        checkpoint = torch.load(comp_root +  selected_model,map_location='cpu')
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        for state in optimizer.state.values():
            for k, v in state.items():
                if isinstance(v, torch.Tensor):
                    state[k] = v.to(device)
        epoch = checkpoint['epoch']
        loss = checkpoint['loss']
        #  model.train()
        if not is_eval:
            model_file_path = comp_root + selected_model
            interim_fig_prev_text = model_file_path[(model_file_path.rfind('_') + 1):(len(model_file_path) - 6)]
            interim_fig_prev = float(interim_fig_prev_text)
            print("using saved model ", model_file_path, " Loss: {:.4f}".format(interim_fig_prev))
    else:
        print("using new model")
    #  finished deciding where the model comes from

    #  For the given model

    #  Print model's state_dict
    #  print("Model's state_dict:")
    #  for param_tensor in model.state_dict():
    #    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

    # Print optimizer's state_dict
    print("Optimizer's state_dict:")
    for var_name in optimizer.state_dict():
        if var_name == "param_groups":
            print(var_name, "\t", optimizer.state_dict()[var_name])
    first_learning_rate(optimizer,learning_rate)
    print("model loaded")

batch_size = batch_sizes

#Load the training set
#train_set = CIFAR10(root="./data", train=True, #transform=train_transformations, download=True)
#Create a loder for the training set
#train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, #num_workers=4)

#Define transformations for the test set
#test_transformations = transforms.Compose([
# transforms.ToTensor(),
# transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#])
# Load the test set, note that train is set to False
#test_set = CIFAR10(root="./data", train=False, transform=test_transformations, #download=True)

# Create a loder for the test set, note that both shuffle is set to false for #the test loader
#test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, #num_workers=4)

loader = HawkLoader(dataPathRoot,batch_size,
                                    pic_size, computer)
train_loader = loader.dataloaderTrain['train']
test_loader = loader.dataloaderTest['val']
train_size = loader.dataset_sizes['train']
test_size = loader.dataset_sizes['val']

# Check if gpu support is available
cuda_avail = torch.cuda.is_available()

# Create model, optimizer and loss function
model = SimpleNet(SimpleNetArgs)

if cuda_avail:
    model.cuda()

optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
loss_fn = nn.CrossEntropyLoss()


# Create a learning rate adjustment function that divides the learning rate by 10 every 30 epochs
def adjust_learning_rate(epoch):
    lr = get_lr(optimizer)

    if epoch == 180:
        lr = lr / 2
    elif epoch == 150:
        lr = lr / 2
    elif epoch == 120:
        lr = lr / 2
    elif epoch == 90:
        lr = lr / 2
    elif epoch == 60:
        lr = lr / 2
    elif epoch == 30:
        lr = lr / 2

    for param_group in optimizer.param_groups:
        param_group["lr"] = lr


#def save_models(epoch,test_corrects):
#    torch.save(model.state_dict(), "cifar10model" + test_corrects +"_{}.model".format(epoch))
#    print("Checkpoint saved")

def save_models(epoch, loss, save_point):
    print("save path types = ",str(type(dataPathRoot))+"\t",str(type(epoch))+"\t",str(type(save_point)))
    save_PATH = dataPathRoot + "/saved_models/" + "Birdies_model_{}_".format(epoch) + "_best_" \
                                + str(save_point) + "_FDpsBSksFn_" + str(no_feature_detectors) + "_" +\
                str(pic_size) + "_" + str(batch_size) + "_" + str(kernel_sizes) + "_" + str(flattener) +".model"
    checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }
    torch.save(checkpoint, save_PATH)
    print("Checkpoint saved")
    if (os.path.exists(save_PATH)):
        print("verified save ", save_PATH)


def test():
    global test_acc, test_acc_abs
    model.eval()
    test_acc_abs = 0
    test_acc = 0.0
    for i, (images, labels) in enumerate(test_loader):

        if cuda_avail:
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())

        # Predict classes using images from the test set
        outputs = model(images)
        _, prediction = torch.max(outputs.data, 1)
        # prediction = prediction.cpu()
        test_acc_abs += torch.sum(prediction == labels.data)

    # Compute the average acc and loss over all 10000 test images
    test_acc = test_acc_abs.cpu().numpy() / test_size
    return (test_acc, test_acc_abs)


def train(num_epochs):
    global best_acc, train_acc, train_loss
    best_acc = 0

    since = time.time()
    for epoch in range(num_epochs):
        model.train()
        train_acc = 0.0
        train_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            # Move images and labels to gpu if available
            if cuda_avail:
                images = Variable(images.cuda())
                labels = Variable(labels.cuda())

            # Clear all accumulated gradients
            optimizer.zero_grad()
            # Predict classes using images from the test set
            outputs = model(images)
            # Compute the loss based on the predictions and actual labels
            #print("outputs ", outputs.shape," labels ",labels.shape)
            loss = loss_fn(outputs, labels)
            # Backpropagate the loss
            loss.backward()

            # Adjust parameters according to the computed gradients
            optimizer.step()

            # train_loss += loss.cpu().data[0] * images.size(0)
            train_loss += loss.cpu().item() * images.size(0)
            _, prediction = torch.max(outputs.data, 1)

            train_acc += torch.sum(prediction == labels.data)

        # Call the learning rate adjustment function
        adjust_learning_rate(epoch)

        # Compute the average acc and loss over all 50000 training images
        train_acc = train_acc.cpu().numpy() / train_size
        train_loss = train_loss / train_size

        # Evaluate on the test set
        results = test()
        test_acc = results[0]
        test_acc_abs = results[1]

            # Save the model if the test acc is greater than our current best
        if test_acc_abs > best_acc and epoch > 1:
                save_models(epoch,loss,str(test_acc_abs.cpu().numpy()))
                best_acc = test_acc_abs

            # Print the metrics
        time_elapsed = time.time() - since
        print("Epoch {}, Train Accuracy: {:.1%} , TrainLoss: {:.4f} , Test Accuracy: {:.1%},"
              "Test Corrects: {}".format(epoch, train_acc, train_loss, test_acc, test_acc_abs),
              ' time {:.0f}h {:.0f}m {:.0f}s'.format(time_elapsed // 3600,(time_elapsed // 60) % 60, time_elapsed % 60))
        # printm()
        


        # View_Test.test(model,deploy_test, validate_path)


if __name__ == "__main__":

    # ------------------------------------------------------------------
    #  fixed prediction == labels.data,
    #-------------------------------------------------------------------
    load_latest_saved_model("Birdies_model_2__best_5_FDpsBSksFn_128_128_64_3_64.model")
    train(200)


looking for  /content/drive/My Drive/Colab Notebooks/saved_models/Birdies_model_2__best_5_FDpsBSksFn_128_128_64_3_64.model
exists =  True
using saved model  /content/drive/My Drive/Colab Notebooks/saved_models/Birdies_model_2__best_5_FDpsBSksFn_128_128_64_3_64.model  Loss: 64.0000
Optimizer's state_dict:
param_groups 	 [{'lr': 0.0001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0.0001, 'amsgrad': False, 'params': [140310872795896, 140310870567024, 140310870567888, 140310871182792, 140312181521936, 140312181519128, 140312181518768, 140312181518408, 140310864146296, 140310864145288, 140310864146080, 140310864145648, 140310864146368, 140310864145936, 140310864146224, 140310864145720, 140310864146008, 140310864144928, 140310830362768, 140310830362912, 140310830363344, 140310830363416, 140310830363560, 140310830363704, 140310830364136, 140310830364208, 140310830364352, 140310830364496, 140310830365000, 140310830365072, 140310830365216, 140310830365360, 140310830365792, 140310830365